# Let's try to find what's wrong with our first implem of :
http://dl.acm.org/citation.cfm?doid=2600428.2609514

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ast import literal_eval
%matplotlib inline

In [75]:
#df = pd.read_csv('data/dummy.csv')
df = pd.read_csv('data/train.csv')
df['recipient_id'] = df['recipient_id'].apply(literal_eval)
#df = df.drop('Unnamed: 0', 1)
X, y = df[['sender_id', 'body']].values, df['recipient_id'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [76]:
df.head()

,sender,sender_id,mid,date,body,recipient_id,recipients
0,christian.yoder@enron.com,104,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,"[125, 126, 127, 117, 128, 129, 130, 131, 132]",robert.badeer@enron.com murray.o neil@enron.co...
1,heather.dunton@enron.com,113,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,"[50, 125, 126, 127, 133, 28, 117, 134, 135, 13...",kim.ward@enron.com robert.badeer@enron.com mur...
2,janel.guerrero@enron.com,49,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,"[125, 146, 147]",robert.badeer@enron.com john.massey@enron.com ...
3,tim.belden@enron.com,117,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,"[125, 130]",robert.badeer@enron.com jeff.richter@enron.com
4,christian.yoder@enron.com,104,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,"[148, 149, 125, 150]",pgillman@schiffhardin.com kamarlantes@calpx.co...


In [77]:
n_train = len(X_train)
n_senders, n_people = 125, 9874
#n_senders, n_people = 2, 15
graph = np.zeros((n_senders, n_people))
for i in range(n_train):
    sender = X_train[i][0]
    for recipient in y_train[i]:
        graph[sender][recipient] += 1
        
total_received = np.sum(graph, axis=0)
# if someone never received message, we put 1 instead of 0 to avoid arthmetical errors...
for i in range(len(total_received)):
    if total_received[i] == 0:
        total_received[i] = 1

graph_n = graph/total_received

First, let's see what happens using only the social graph

In [78]:
sender_favorites = graph.argsort()[:,-10:][:,::-1]

In [79]:
y_pred = sender_favorites[X_test[:,0].astype(int)]

In [80]:
graph_n.shape

(125, 9874)

In [81]:
def ap(recommanded, real):
    real_set = set(real)
    cur = 0.
    n = len(recommanded)
    ans = 0.
    for k in range(1, n+1):
        if recommanded[k-1] in real_set:
            cur += 1
            ans += cur/k
    return ans/min(n, len(real))

def MAP(recommanded, real):
    ans = 0.
    for i in range(len(recommanded)):
        ans += ap(recommanded[i], real[i])
    return ans/len(recommanded)

MAP(y_pred, y_test)

0.2990863886465465